# Exploration Gold

### 1. Exploration des données Gold en Notebook

Voici une cellule optimisée pour inspecter ton fichier Master et tes KPIs. On utilise DuckDB pour garder la même logique que ton pipeline

In [ ]:
import duckdb
import pandas as pd
from pathlib import Path
from sirene_pipeline.config import settings

# 1. Setup paths (handling the notebooks/ directory offset)
root_path = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
gold_dir = root_path / settings.gold.output_dir

con = duckdb.connect()

# 2. Check the Master Table (The enriched one)
master_file = gold_dir / settings.gold.master_filename
if master_file.exists():
    print(f"✅ Loading Master Table: {master_file.name}")
    con.execute(f"CREATE OR REPLACE VIEW master AS SELECT * FROM read_parquet('{master_file.as_posix()}')")
    
    # Show a few rows to check the join result
    display(con.execute("""
        SELECT siret, denominationUniteLegale, categorieEntreprise, departement, secteur_activite 
        FROM master 
        LIMIT 5
    """).df())
else:
    print("❌ Master file not found. Did you run the Gold job?")

# 3. Dynamic check of all Gold KPIs
print(f"\n--- Checking all KPIs in {gold_dir} ---")

# We iterate over the KPI dictionary defined in settings.toml
for kpi_name, filename in settings.gold.kpis.items():
    kpi_file = gold_dir / filename
    
    print(f"\n📊 KPI: {kpi_name} ({filename})")
    
    if kpi_file.exists():
        # Read and display the first few rows
        df_kpi = pd.read_parquet(kpi_file)
        print(f"✅ Found {len(df_kpi)} rows.")
        display(df_kpi.head())
    else:
        print(f"❌ File not found. Make sure the Gold job has finished.")

✅ Loading Master Table: sirene_master_enriched.parquet


,siret,denominationUniteLegale,categorieEntreprise,departement,secteur_activite
0,29750055500018,None,None,75,42
1,29750061300015,None,None,75,42
2,29750068800017,None,None,75,42
3,29750082900017,None,None,75,42
4,29750122300012,None,None,75,42



✅ Checking KPI: kpi_dominant_sectors.parquet


,departement,secteur_activite,count
0,75,42,103
1,92,55,1
2,77,70,272
3,94,74,1
4,78,42,40
